In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer
from sklearn.metrics import roc_curve, roc_auc_score, auc, ConfusionMatrixDisplay, classification_report, confusion_matrix
from sklearn.inspection import permutation_importance
from sksurv.metrics import concordance_index_censored, concordance_index_ipcw, brier_score

import xgboost as xgb

from lifelines.statistics import logrank_test
from lifelines import KaplanMeierFitter

In [ ]:
def train_preprocessing(df, ohe_encoder=None, normalizer='StandardScaler'):
    """
    Preprocesses the dataset for machine learning model training.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the data to be preprocessed.
    ohe_encoder : list, optional
        List of categorical columns to be transformed with One-Hot Encoding.
        If `None`, this step will be skipped. Default is `None`.
    normalizer : str, optional
        Normalization method to be applied. Choose between:
        - 'StandardScaler': Standard normalization (mean=0, std=1).
        - 'MinMaxScaler': Scales data to range [0, 1].
        - 'MaxAbsScaler': Scales data by maximum absolute value.
        - 'QuantileTransformer': Transforms data to normal distribution.
        Default is 'StandardScaler'.

    Returns:
    --------
    df_aux : numpy.ndarray
        Numpy matrix containing the preprocessed and normalized data.
    enc : dict
        Dictionary containing the encoders used for categorical columns.
        May include `OneHotEncoder` and `OrdinalEncoder`.
    norm : sklearn.preprocessing.TransformerMixin
        Normalizer object used (such as `StandardScaler`, `MinMaxScaler`, etc.).
    feat_cols : pandas.Index
        List of columns/features present after preprocessing.

    Notes:
    ------
    - Columns specified in `ohe_encoder` are transformed using `OneHotEncoder`.
    - Remaining categorical columns are encoded with `OrdinalEncoder`.
    - Normalization is applied to all columns after categorical encoding.
    """
    # Data Transformation: One-Hot Encoding, Ordinal Encoding, and Normalization
    df_aux = df.copy()  # Create a copy of the input DataFrame

    enc = dict()  # Initialize a dictionary to store encoders

    # Apply One-Hot Encoding (if specified)
    if ohe_encoder is not None:
        for col in ohe_encoder:  # Iterate through columns to be one-hot encoded
            enc[col] = OneHotEncoder(handle_unknown='ignore', drop='first')  # Initialize and fit OneHotEncoder
            ohe_results = enc[col].fit_transform(df_aux[[col]])  # Apply one-hot encoding and transform
            df1 = pd.DataFrame(ohe_results.toarray(),  # Create DataFrame from encoded results
                            columns=enc[col].get_feature_names_out(),
                            index=df_aux[col].index)
            df_aux = df_aux.merge(df1, how='left', left_index=True, right_index=True)  # Merge encoded columns into main DataFrame

        df_aux.drop(columns=ohe_encoder, inplace=True)  # Drop original one-hot encoded columns


    # Apply Ordinal Encoding to remaining categorical columns
    list_categorical = df_aux.select_dtypes(include='object').columns  # Identify categorical columns
    for col in list_categorical:  # Iterate through categorical columns
        enc[col] = OrdinalEncoder(handle_unknown='use_encoded_value',  # Initialize and fit OrdinalEncoder
                                unknown_value=-1)  # Handle unknown values by assigning -1
        df_aux[col] = enc[col].fit_transform(df_aux[[col]])  # Apply ordinal encoding and transform

    feat_cols = df_aux.columns  # Store the feature names after encoding

    # Apply Normalization based on the specified method
    if normalizer == 'StandardScaler':
        norm = StandardScaler()
    elif normalizer == 'MinMaxScaler':
        norm = MinMaxScaler((0, 1))
    elif normalizer == 'MaxAbsScaler':
        norm = MaxAbsScaler()
    elif normalizer == 'QuantileTransformer':
        norm = QuantileTransformer(output_distribution='normal')

    df_aux = norm.fit_transform(df_aux)  # Fit and transform the data using the selected scaler

    return df_aux, enc, norm, feat_cols  # Return the transformed DataFrame, encoders, normalizer, and feature names

################################################################################

def test_preprocessing(df, enc, norm, ohe_encoder=None):
    """
    Preprocesses the test dataset using the transformers fitted on the training set.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the test data to be preprocessed.
    enc : dict
        Dictionary containing the encoders fitted on the training set.
        May include `OneHotEncoder` and `OrdinalEncoder` for categorical columns.
    norm : sklearn.preprocessing.TransformerMixin
        Normalizer object fitted on the training set (such as `StandardScaler`, `MinMaxScaler`, etc.).
    ohe_encoder : list, optional
        List of categorical columns that were transformed with One-Hot Encoding during training.
        If `None`, this step will be skipped. Default is `None`.

    Returns:
    --------
    df_aux : numpy.ndarray
        Numpy matrix containing the preprocessed and normalized test data.

    Notes:
    ------
    - Columns specified in `ohe_encoder` are transformed using the `OneHotEncoder` fitted on the training set.
    - Remaining categorical columns are transformed using the `OrdinalEncoder` fitted on training.
    - Normalization is applied to all columns after categorical encoding.
    - It's important to ensure that `enc` and `norm` are the same ones used in preprocessing the training set.
    """
    # Data Transformation: One-Hot Encoding and Normalization
    df_aux = df.copy()  # Create a copy of the input DataFrame

    # Apply One-Hot Encoding (if encoder is provided)
    if ohe_encoder is not None:
        for col in ohe_encoder: # Iterate through categorical columns to be encoded
            ohe_results = enc[col].transform(df_aux[[col]])  # Apply one-hot encoding
            df1 = pd.DataFrame(ohe_results.toarray(),  # Create a new DataFrame with the encoded features
                            columns=enc[col].get_feature_names_out(),
                            index=df_aux[col].index)
            df_aux = df_aux.merge(df1, how='left', left_index=True, right_index=True)  # Merge encoded features into the main DataFrame

        df_aux.drop(columns=ohe_encoder, inplace=True)  # Remove original categorical columns

    # Apply Encoding to other categorical features
    list_categorical = df_aux.select_dtypes(include='object').columns #Get the list of categorical columns
    for col in list_categorical: #Iterate through the categorical columns
        df_aux[col] = enc[col].transform(df_aux[[col]]) #Apply the transformation

    # Apply Normalization
    df_aux = norm.transform(df_aux)  # Apply normalization to the entire DataFrame

    return df_aux  # Return the transformed DataFrame

################################################################################

def preprocessing(x_train, x_test, ohe_encoder=None, norm_name='StandardScaler',
                  return_enc_norm=False, random_state=0):
    """
    Preprocesses both training and test datasets, including category encoding and normalization.

    Parameters:
    -----------
    x_train : pandas.DataFrame
        Training dataset.
    x_test : pandas.DataFrame
        Test dataset.
    ohe_encoder : list, optional
        List of categorical columns to be transformed with One-Hot Encoding.
        If `None`, this step will be skipped. Default is `None`.
    norm_name : str, optional
        Name of the normalization method to be applied. Choose between:
        - 'StandardScaler': Standard normalization (mean=0, std=1).
        - 'MinMaxScaler': Scales data to range [0, 1].
        - 'MaxAbsScaler': Scales data by maximum absolute value.
        - 'QuantileTransformer': Transforms data to normal distribution.
        Default is 'StandardScaler'.
    return_enc_norm : bool, optional
        If `True`, also returns the fitted `enc` (encoders) and `norm` (normalizer) objects.
        Default is `False`.
    random_state : int, optional
        Seed to ensure reproducibility in preprocessing.
        Default is `0`.

    Returns:
    --------
    X_train_ : numpy.ndarray
        Numpy matrix containing the preprocessed and normalized training data.
    X_test_ : numpy.ndarray
        Numpy matrix containing the preprocessed and normalized test data.
    feat_cols : pandas.Index
        List of columns/features present after preprocessing.
    enc : dict, optional
        Dictionary containing the encoders fitted on the training set (returned if `return_enc_norm=True`).
    norm : sklearn.preprocessing.TransformerMixin, optional
        Normalizer object fitted on the training set (returned if `return_enc_norm=True`).

    Notes:
    ------
    - The function internally uses `train_preprocessing` to fit the encoders and normalizers on the training set.
    - The test set is transformed using the same encoders and normalizers fitted on training.
    - If `return_enc_norm=False`, only the preprocessed data and feature columns are returned.
    """
    # Data Preprocessing
    # Apply preprocessing steps to the training and testing data
    X_train_, enc, norm, feat_cols = train_preprocessing(x_train,
                                                         ohe_encoder=ohe_encoder,
                                                         normalizer=norm_name)
    X_test_ = test_preprocessing(x_test, enc, norm, ohe_encoder=ohe_encoder)

    # Return preprocessed data and preprocessing objects (encoder and normalizer) if requested
    if return_enc_norm:
        return X_train_, X_test_, feat_cols, enc, norm
    else:
        return X_train_, X_test_, feat_cols

################################################################################

def plot_survival_curve(x, y, name, compare_km=False, test_df=None):
    """
    Plots the survival curve for the provided data.

    Parameters:
    -----------
    x : array-like
        Vector representing the analysis times.
    y : array-like
        Vector representing the survival probabilities associated with each time in `x`.
    name : str
        Name or label for the survival curve.
    compare_km : bool, optional
        If True, adds the Kaplan-Meier (KM) curve to the plot for comparison with the provided curve.

    Returns:
    --------
    None
        The function doesn't return any value, but displays a plot of the survival curve.

    Notes:
    ------
    - If `compare_km=True`, the Kaplan-Meier curve will be calculated using the variables
      `test_df.obito_geral` (event indicator) and `test_df.meses_diag` (follow-up time).
    - The plot displays the survival function on the y-axis and time (in months) on the x-axis.
    """
    # Survival Curve Plotting
    fig = plt.figure(figsize=(12, 6))  # Create the plot figure
    ax = plt.subplot(111)  # Create the axes for the plot

    # Kaplan-Meier Comparison (if enabled)
    if compare_km:
        E = (test_df.obito_geral == 1)  # Extract event data (death)
        T = test_df.meses_diag  # Extract time-to-event data (months since diagnosis)

        kmf = KaplanMeierFitter()  # Initialize Kaplan-Meier model
        ax = kmf.fit(T, E).plot_survival_function(ax=ax, ci_show=False,  # Fit and plot Kaplan-Meier survival curve
                                                show_censors=True,  # Show censoring events
                                                label='Kaplan-Meier')  # Set label for the curve
        title = f'Kaplan Meier vs {name}'  # Set title for comparison plot

    else:
        title = name  # Set title if no comparison

    # Plotting Model Survival Curve
    ax.step(x, y, where='post', label=name)  # Plot the model's survival curve

    # Plot Formatting
    ax.set_title(title)  # Set the plot title
    ax.set_xlabel('Months after diagnosis')  # Set x-axis label
    ax.set_ylabel('Survival function')  # Set y-axis label
    ax.spines['top'].set_visible(False)  # Remove top border
    ax.spines['right'].set_visible(False)  # Remove right border
    plt.xticks(np.arange(0, x[-1] + 11, 12))  # Set x-axis ticks every 12 months
    plt.ylim([0, 1.1])  # Set y-axis limits
    plt.legend();  # Show the legend

################################################################################

def c_index_ipcw_brier_score(model, X_train, y_train, X_test, y_test):
    """
    Calculates the IPCW Concordance Index, time-dependent Brier Score, and Integrated Brier Score (IBS) for a survival model.

    Parameters:
    -----------
    model : object
        Trained survival model that supports `predict` (risk) and
        `predict_survival_function` (survival curves) methods.
    X_train : array-like
        Input features of the training set.
    y_train : pandas.DataFrame
        Training set containing `event` (event indicator) and `time` (time to event or censoring) columns.
    X_test : array-like
        Input features of the test set.
    y_test : pandas.DataFrame
        Test set containing `event` (event indicator) and `time` (time to event or censoring) columns.

    Returns:
    --------
    None
        The function doesn't return values directly, but displays the following results in the console and plots:
        - IPCW C-Index for training and test sets.
        - Time-dependent Brier Score.
        - Integrated Brier Score (IBS).
        - Brier Scores at 1, 3, and 5 years.
        - Plot of the Brier Score over time.

    Notes:
    ------
    - The IPCW (Inverse Probability of Censoring Weighting) C-Index is an adjusted version of the concordance index to handle censored data.
    - The Brier Score measures prediction accuracy over time. The IBS is the integral of the Brier Score over the entire follow-up period.
    - Model predictions are based on risk and survival curves for the test set.
    """
    # Model Evaluation: Predictions and Concordance Index
    # Generate predictions (risk or survival curves) for train and test sets
    surv_risks = model.predict(X_test)
    surv_risks_train = model.predict(X_train)

    # Calculate the IPCW Concordance Index for both test and train sets
    c_index_ipcw = concordance_index_ipcw(y_train, y_test, surv_risks)
    c_index_ipcw_train = concordance_index_ipcw(y_train, y_train, surv_risks_train)
    print(f'C-Index IPCW Test: {c_index_ipcw[0]}')
    print(f'C-Index IPCW Train: {c_index_ipcw_train[0]}')


    # Model Evaluation: Brier Score and IBS
    # Predict survival curves for the test set
    surv_curves = model.predict_survival_function(X_test, return_array=True)

    # Calculate the Brier Score at various time points
    time_points = model.unique_times_[:-1]  # Define the time points for evaluation
    brier_scores = brier_score(y_train, y_test, surv_curves[:, :-1], time_points)

    # Calculate the Integrated Brier Score (IBS)
    time_differences = np.diff(np.hstack([[0], time_points]))
    ibs = np.sum(brier_scores[1] * time_differences) / time_points[-1]
    print(f'\nIntegrated Brier Score (IBS): {ibs}')

    # Print Brier Score at specific time points (1, 3, and 5 years)
    print(f'\nBrier Score:')
    print(f'    > 1 year = {brier_scores[1][11]}')
    print(f'    > 3 years = {brier_scores[1][35]}')
    print(f'    > 5 years = {brier_scores[1][59]}\n')

    # Plotting the Brier Score
    plt.figure(figsize=(10, 6))  # Create the plot figure
    plt.plot(brier_scores[0], brier_scores[1], label='Brier Score')  # Plot the Brier Score over time
    plt.xlabel('Time (months)')  # Set x-axis label
    plt.ylabel('Brier Score')  # Set y-axis label
    # plt.title('Brier Score over time')  # Optional plot title
    plt.legend()  # Show the plot legend
    plt.show()  # Display the plot

################################################################################

def media_mediana(time_bins, surv_curves, is_xgbse=False):
    """
    Calculates and displays survival statistics (mean and median) based on the provided survival curves.

    Parameters:
    -----------
    time_bins : array-like
        Vector containing the times corresponding to the analysis intervals of survival curves.
    surv_curves : array-like
        Matrices or lists containing survival curves for each individual or group.
    is_xgbse : bool, optional
        Indicates whether the survival curves were generated by the XGBSE model. If True, curves are converted to NumPy arrays.

    Returns:
    --------
    None
        The function doesn't return any value, but prints:
        - The number of patients with survival probability below 50% at some point.
        - The mean and median survival times for a 50% probability.
        - The times for mean and median survival curves.

    Notes:
    ------
    - The median is the time when survival probability reaches or falls below 50%.
    - Mean and median curves are calculated by aggregating the provided curves.
    """
    # Calculate survival times for individual curves
    surv_times = []

    if is_xgbse:
        surv_curves = surv_curves.values

    for curve in surv_curves:
        id = np.where(curve <= 0.5)[0]  # Find indices where survival probability is <= 50%

        if len(id) > 0:
            surv_times.append(time_bins[id[0]])  # Append the corresponding time

    print(f'Number of patients with less than 50% survival at some point: {len(surv_times)}')

    print(f'    Mean: {np.mean(surv_times):.2f} Months')  # Print mean survival time
    print(f'    Median: {np.median(surv_times):.2f} Months')  # Print median survival time


    # Calculate and print mean and median survival times from aggregated curves
    surv_mean = surv_curves.mean(axis=0)  # Calculate mean survival curve
    id_media = np.where(surv_mean <= 0.5)[0]  # Find index where mean survival probability is <= 50%
    if len(id_media) > 0:
        media = time_bins[id_media[0]]  # Get time from mean curve
    else:
        media = 0

    surv_median = np.median(surv_curves, axis=0)  # Calculate median survival curve
    id_median = np.where(surv_median <= 0.5)[0]  # Find index where median survival probability is <= 50%
    if len(id_median) > 0:
        mediana = time_bins[id_median[0]]  # Get time from median curve
    else:
        mediana = 0

    print(f'\nTimes of the mean and median curves:')
    print(f'    Mean: {media:.2f} Months')  # Print mean survival time
    print(f'    Median: {mediana:.2f} Months')  # Print median survival time

################################################################################

def c_index_scorer(estimator, X, y):
    """
    Calculates the Concordance Index (C-Index) for a survival model.

    Parameters:
    -----------
    estimator : object
        Trained estimator model. It must support the `predict_survival_function` or `predict` methods.
    X : array-like
        Input data for model predictions.
    y : pandas.DataFrame
        Output data containing `event` (event indicator) and `time` (time to event or censoring) columns.

    Returns:
    --------
    c_index : float
        Concordance Index (C-Index) value, ranging from 0 to 1.
        Values closer to 1 indicate better discrimination ability of the model.

    Notes:
    ------
    - The function attempts to use the estimator's `predict_survival_function` method to obtain survival curve predictions.
      If this method is unavailable, it uses the `predict` method to obtain risk scores.
    - The C-Index evaluates the model's ability to correctly rank relative survival times between samples.
    - Ensure that the `y` object is correctly formatted, with the necessary columns.
    """
    try:
        # Try to use the `predict_survival_function` method (preferred if available and returns numpy array)
        y_pred = estimator.predict_survival_function(X, return_array=True)

        # Check if y_pred is a numpy array; if not, try predict
        if not isinstance(y_pred, np.ndarray):
            raise AttributeError("predict_survival_function did not return a NumPy array; falling back to predict")

    except AttributeError:
        # If the `predict_survival_function` method does not exist or does not return numpy array, use the `predict` method (risk scores)
        y_pred = estimator.predict(X)

    # Calculate and return the C-Index
    # The concordance_index function expects risk scores where higher values = shorter survival
    c_index = concordance_index(y['event'], y['time'], y_pred)  # Pass event and time separately

    return c_index

################################################################################

def c_index_metric(y_true, y_pred):
    """
    Calculates the Concordance Index (C-Index) to evaluate survival models.

    Parameters:
    -----------
    y_true : pandas.DataFrame
        DataFrame containing the ground truth survival data.
        It must include the following columns:
        - `event` : bool
            Event indicator (True for event occurred, False for censoring).
        - `time` : float
            Time to event or censoring.
    y_pred : array-like
        Model predicted values, representing risks or another related metric.

    Returns:
    --------
    c_index : float
        Concordance Index (C-Index) value, ranging from 0 to 1.
        Values closer to 1 indicate better discrimination ability of the model.

    Notes:
    ------
    - The Concordance Index measures the model's ability to correctly rank relative survival times
      between samples.
    - For a reliable C-Index, `y_pred` should reflect the ordering of risks or expected survival times.
    """
    # Extract event and time information from y_true
    event = y_true['event']
    time = y_true['time']

    # Calculate the C-Index.
    c_index = concordance_index_censored(event, time, y_pred)[0]  # [0] gets the c-index, other values are related to ties.

    return c_index

################################################################################

def c_index_metric_lgbm(y_true, y_pred):
    """
    Calculates the Concordance Index (C-Index) to evaluate survival models
    trained with LightGBM.

    Parameters:
    -----------
    y_true : pandas.DataFrame
        DataFrame containing the ground truth survival data.
        It must include the following columns:
        - `event` : bool
            Event indicator (True for event occurred, False for censoring).
        - `time` : float
            Time to event or censoring.
    y_pred : array-like
        Values predicted by the LightGBM model, representing risks
        (higher values indicate greater risk).

    Returns:
    --------
    c_index : float
        Concordance Index (C-Index) value, ranging from 0 to 1.
        Values closer to 1 indicate better discrimination ability of the model.

    Notes:
    ------
    - LightGBM returns risks directly, but to calculate the C-Index, we use the negative of the risks (`-y_pred`),
      since the C-Index assumes that higher values correspond to longer survival times.
    - Ensure that `y_pred` is aligned with the data in `y_true`.
    """
    # Extract event and time information from y_true
    event = y_true['event']
    time = y_true['time']

    # Calculate the C-Index. Note the negation of y_pred as higher values = greater risk
    c_index = concordance_index_censored(event, time, -y_pred)[0]  # [0] gets the c-index, other values are related to ties.

    return c_index

################################################################################

def calculate_permutation_importance(model, X_test, y_test, feat_cols,
                                     n_repeats=10, random_state=42):
    """
    Calculates feature importance of a model using the permutation technique.

    Permutation importance is calculated as the reduction in model performance (using C-Index)
    when randomly shuffling the values of a feature in the test set.

    Parameters:
    -----------
    model : object
        Trained model that implements the `predict` method. The model should predict risks
        or related values for survival evaluation.
    X_test : pandas.DataFrame or numpy.ndarray
        Test dataset containing the features.
        If it's a NumPy array, it will be converted to a DataFrame using `feat_cols` as column names.
    y_test : pandas.DataFrame
        DataFrame containing the ground truth survival data:
        - Column `event`: indicates whether the event occurred (1) or was censored (0).
        - Column `time`: time to event or censoring.
    feat_cols : list
        List with the names of the features present in `X_test`.
    n_repeats : int, optional
        Number of repetitions for the permutation of each feature. Defaults to 10.
    random_state : int, optional
        Seed for random number generation, ensuring reproducibility.

    Returns:
    --------
    importance_df : pandas.DataFrame
        DataFrame containing the means and standard deviations of the calculated importances
        for each feature.
        The columns include:
        - `importances_mean`: mean of the permutation importances.
        - `importances_std`: standard deviation of the permutation importances.

    Notes:
    ------
    - The baseline is calculated using the C-Index on the test set without permutation.
    - Shuffling a feature reduces the information contained in the data, resulting in an expected
      decrease in performance.
    - The method is suitable for survival models that use the C-Index as a metric.
    """
    # Ensure X_test is a DataFrame
    if isinstance(X_test, np.ndarray):
        X_test = pd.DataFrame(X_test, columns=feat_cols)

    # Predict initial risks on the test set.  Using .values to get the numpy array
    y_pred = model.predict(X_test.values)  # Use .values for numpy array input

    # Calculate the baseline C-Index (no permutation)
    baseline_c_index = c_index_metric(y_test, y_pred)

    # Initialize lists to store the results
    importances_mean = []
    importances_std = []

    # Iterate over each feature to calculate permutation importance
    rng = np.random.RandomState(random_state)
    for col in feat_cols:
        scores = []

        for _ in range(n_repeats):  # Repeat permutation
            X_test_permuted = X_test.copy()

            # Permute the current column
            X_test_permuted[col] = rng.permutation(X_test[col].values)

            # Predict again with the permuted column. Use .values for numpy array input
            y_pred_permuted = model.predict(X_test_permuted.values)  # Use .values for numpy array input

            # Calculate the new C-Index
            permuted_c_index = c_index_metric(y_test, y_pred_permuted)

            # Record the difference in C-Index
            scores.append(baseline_c_index - permuted_c_index)

        # Store the mean and standard deviation of the importances
        importances_mean.append(np.mean(scores))
        importances_std.append(np.std(scores))

    # Create a DataFrame with the results
    importance_df = pd.DataFrame({
        'importances_mean': importances_mean,
        'importances_std': importances_std
    }, index=feat_cols).sort_values(by='importances_mean', ascending=False)

    return importance_df

################################################################################

def calculate_permutation_importance_xgb(model, X_test, y_test, feat_cols,
                                         n_repeats=10, random_state=42):
    """
    Calculates feature importance of an XGBoost model using the permutation technique.

    Permutation importance is calculated as the reduction in model performance (using C-Index)
    when randomly shuffling the values of a feature in the test set.

    Parameters:
    -----------
    model : object
        Trained XGBoost model that implements the `predict` method. The model should predict risks
        or related values for survival evaluation.
    X_test : pandas.DataFrame or numpy.ndarray
        Test dataset containing the features.
        If it's a NumPy array, it will be converted to a DataFrame using `feat_cols` as column names.
    y_test : pandas.DataFrame
        DataFrame containing the ground truth survival data:
        - Column `event`: indicates whether the event occurred (1) or was censored (0).
        - Column `time`: time to event or censoring.
    feat_cols : list
        List with the names of the features present in `X_test`.
    n_repeats : int, optional
        Number of repetitions for the permutation of each feature. Defaults to 10.
    random_state : int, optional
        Seed for random number generation, ensuring reproducibility.

    Returns:
    --------
    importance_df : pandas.DataFrame
        DataFrame containing the means and standard deviations of the calculated importances
        for each feature.
        The columns include:
        - `importances_mean`: mean of the permutation importances.
        - `importances_std`: standard deviation of the permutation importances.

    Notes:
    ------
    - The baseline is calculated using the C-Index on the test set without permutation.
    - Shuffling a feature reduces the information contained in the data, resulting in an expected
      decrease in performance.
    - The method is suitable for survival models that use the C-Index as a metric.
    - The model should be trained to predict survival times or risks, rather than a simple classification.
    """
    # Ensure X_test is a DataFrame
    if isinstance(X_test, np.ndarray):
        X_test = pd.DataFrame(X_test, columns=feat_cols)

    # Predict initial risks on the test set using a DMatrix for XGBoost
    dtest = xgb.DMatrix(X_test)  # Create DMatrix for efficiency with XGBoost
    y_pred = model.predict(dtest)

    # Calculate the baseline C-Index (no permutation)
    baseline_c_index = c_index_metric(y_test, y_pred)

    # Initialize lists to store the results
    importances_mean = []
    importances_std = []

    # Iterate over each feature to calculate permutation importance
    rng = np.random.RandomState(random_state)
    for col in feat_cols:
        scores = []

        for _ in range(n_repeats):  # Repeat permutation
            X_test_permuted = X_test.copy()

            # Permute the current column
            X_test_permuted[col] = rng.permutation(X_test[col].values)

            # Predict again with the permuted column using a DMatrix
            dtest_permuted = xgb.DMatrix(X_test_permuted) # Create DMatrix for permuted data
            y_pred_permuted = model.predict(dtest_permuted)

            # Calculate the new C-Index
            permuted_c_index = c_index_metric(y_test, y_pred_permuted)

            # Record the difference in C-Index
            scores.append(baseline_c_index - permuted_c_index)

        # Store the mean and standard deviation of the importances
        importances_mean.append(np.mean(scores))
        importances_std.append(np.std(scores))

    # Create a DataFrame with the results
    importance_df = pd.DataFrame({
        'importances_mean': importances_mean,
        'importances_std': importances_std
    }, index=feat_cols).sort_values(by='importances_mean', ascending=False)

    return importance_df

################################################################################

def calculate_permutation_importance_lgbm(model, X_test, y_test, feat_cols,
                                          n_repeats=10, random_state=42):
    """
    Calculates feature importance of a LightGBM model using the permutation technique.

    Permutation importance is calculated as the reduction in model performance (using C-Index)
    when randomly shuffling the values of a feature in the test set.

    Parameters:
    -----------
    model : object
        Trained LightGBM model that implements the `predict` method. The model should predict risks
        or related values for survival evaluation.
    X_test : pandas.DataFrame or numpy.ndarray
        Test dataset containing the features.
        If it's a NumPy array, it will be converted to a DataFrame using `feat_cols` as column names.
    y_test : pandas.DataFrame
        DataFrame containing the ground truth survival data:
        - Column `event`: indicates whether the event occurred (1) or was censored (0).
        - Column `time`: time to event or censoring.
    feat_cols : list
        List with the names of the features present in `X_test`.
    n_repeats : int, optional
        Number of repetitions for the permutation of each feature. Defaults to 10.
    random_state : int, optional
        Seed for random number generation, ensuring reproducibility.

    Returns:
    --------
    importance_df : pandas.DataFrame
        DataFrame containing the means and standard deviations of the calculated importances
        for each feature.
        The columns include:
        - `importances_mean`: mean of the permutation importances.
        - `importances_std`: standard deviation of the permutation importances.

    Notes:
    ------
    - The baseline is calculated using the C-Index on the test set without permutation.
    - Shuffling a feature reduces the information contained in the data, resulting in an expected
      decrease in performance.
    - The method is suitable for survival models that use the C-Index as a metric.
    - The model should be trained to predict survival times or risks, rather than a simple classification.
    """
    # Ensure X_test is a DataFrame
    if isinstance(X_test, np.ndarray):
        X_test = pd.DataFrame(X_test, columns=feat_cols)

    # Predict initial risks on the test set
    y_pred = model.predict(X_test)

    # Calculate the baseline C-Index (no permutation)
    baseline_c_index = c_index_metric_lgbm(y_test, y_pred)

    # Initialize lists to store the results
    importances_mean = []
    importances_std = []

    # Iterate over each feature to calculate permutation importance
    rng = np.random.RandomState(random_state)
    for col in feat_cols:
        scores = []

        for _ in range(n_repeats):  # Repeat permutation
            X_test_permuted = X_test.copy()

            # Permute the current column
            X_test_permuted[col] = rng.permutation(X_test[col].values)

            # Predict again with the permuted column
            y_pred_permuted = model.predict(X_test_permuted)

            # Calculate the new C-Index
            permuted_c_index = c_index_metric_lgbm(y_test, y_pred_permuted)

            # Record the difference in C-Index
            scores.append(baseline_c_index - permuted_c_index)

        # Store the mean and standard deviation of the importances
        importances_mean.append(np.mean(scores))
        importances_std.append(np.std(scores))

    # Create a DataFrame with the results
    importance_df = pd.DataFrame({
        'importances_mean': importances_mean,
        'importances_std': importances_std
    }, index=feat_cols).sort_values(by='importances_mean', ascending=False)

    return importance_df

################################################################################

def predict_survival_probability(surv_risks, survival_prob, time):
    """
    Calculates the adjusted survival probability for each individual at each time point.
    """
    # Calculate the baseline hazard from the survival probability
    baseline_hazard = -np.log(survival_prob)  # Vector of size (n_time,)

    # Align dimensions: expand surv_risks to (n_individuals, 1)
    surv_risks = surv_risks[:, np.newaxis]  # Now (n_individuals, 1)

    # Multiply the risks (n_individuals, 1) by the baseline hazard (1, n_time)
    survival_probabilities = np.exp(-surv_risks * baseline_hazard)  # Now (n_individuals, n_time)

    return survival_probabilities